In [45]:
!pip install pulp

In [46]:
import pulp
import numpy as np
print("PuLP version:", pulp.__version__)
!python -V

PuLP version: 2.5.1
Python 3.7.12


## Sample Average Approximaiton (SAA)

In [63]:
N = 30  # sample size
M = 15  # batches

X = np.zeros([M, 3])
z = np.zeros([M, 1])
for m in range(M):
  delta = np.random.normal(1, 0.1, N)  # Random Sampling
  avg_yield = [2.5, 3, 20]

  # Initialize Class
  lp = pulp.LpProblem("Maximize_Profits", pulp.LpMaximize)

  # Define Decicison Variables
  ## 1st stage
  x1 = pulp.LpVariable(name="wheat_area", lowBound=0)
  x2 = pulp.LpVariable(name="corn_area", lowBound=0)
  x3 = pulp.LpVariable(name="sugar_beet_area", lowBound=0)
  ## 2nd stage
  var_keys = np.arange(N)
  Y_1 = pulp.LpVariable.dict("wheat_buy", var_keys, lowBound=0)
  Y_2 = pulp.LpVariable.dict("corn_buy", var_keys, lowBound=0)
  Z_1 = pulp.LpVariable.dict("wheat_sell", var_keys, lowBound=0)
  Z_2 = pulp.LpVariable.dict("corn_sell", var_keys, lowBound=0)
  Z_3 = pulp.LpVariable.dict("sugar_beet_sellH", var_keys, lowBound=0)
  Z_4 = pulp.LpVariable.dict("sugar_beet_sellL", var_keys, lowBound=0)

  # Define Objective Function
  coeff_dict = dict(zip(np.arange(6), [-238, -210, 170, 150, 36, 10]))
  rv_dict = dict(zip(np.arange(6), [Y_1, Y_2, Z_1, Z_2, Z_3, Z_4]))
  lp += -150*x1 - 230*x2 - 260*x3 + \
      pulp.lpSum(pulp.lpSum(coeff_dict[j]*rv_dict[j][i]
                for j in np.arange(6)) for i in var_keys) / N

  # Define Constraints
  lp += x1 + x2 + x3 <= 500
  for i in var_keys:
      lp += delta[i]*avg_yield[0]*x1 + Y_1[i] - Z_1[i] >= 200
      lp += delta[i]*avg_yield[1]*x2 + Y_2[i] - Z_2[i] >= 240
      lp += Z_3[i] + Z_4[i] <= delta[i]*avg_yield[2]*x3
      lp += Z_3[i] <= 6000

  # Solve Model
  lp.solve()
  profits = pulp.value(lp.objective)
  X[m, 0] = pulp.value(x1)
  X[m, 1] = pulp.value(x2)
  X[m, 2] = pulp.value(x3)
  z[m, 0] = profits
  # print(f"{str(x1):>20} =", pulp.value(x1))
  # print(f"{str(x2):>20} =", pulp.value(x2))
  # print(f"{str(x3):>20} =", pulp.value(x3))
  # print("\nProfits =",profits)

# Calculate lower bound
L_MN = np.average(z)
var_l = sum(np.square(z - L_MN))[0]/(M - 1)
print(f"L_MN  = {L_MN}")
print(f"var_l = {var_l}")

L_MN  = 111552.36461311654
var_l = 11633432.487356598


In [ ]:
# Print all batches
# print(f"sample_size={N} batches={M}\n")
# print("Resource X:\n", X)
# print("Profits Z:\n", z)